In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.0.2
3.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Expected Annual Damage', variable_name=('A.1_Expected Annual Damage',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.1 Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.2 Expected Annual Damage', variable_name=('A.2_Expected Annual Damage',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.2 Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.3 Expected Annual Damage', variable_name=('A.3_Expected Annual Damage',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.3 Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOutcome('A.4 Expected Annual Damage', variable_name=('A.4_Expected Annual Damage',), function=<function sum_over at 0x7f63d59122a0>)
ScalarOut

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:24<00:00,  8.16it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Expected Annual Damage', 'A.1 Expected Number of Deaths', 'A.2 Expected Annual Damage', 'A.2 Expected Number of Deaths', 'A.3 Expected Annual Damage', 'A.3 Expected Number of Deaths', 'A.4 Expected Annual Damage', 'A.4 Expected Number of Deaths', 'A.5 Expected Annual Damage', 'A.5 Expected Number of Deaths', 'Total Infrastructure Costs', 'Total Expected Annual Damage', 'Total Expected Number of Deaths'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,44,215.312098,10.0,0.816761,246.341340,1.0,0.986961,138.292142,1.5,0.408531,...,6,3,6,8,7,3,4,4,0,dikesnet
1,86,190.087673,10.0,0.476498,333.397444,1.0,0.708017,337.632420,1.5,0.342848,...,6,3,6,8,7,3,4,5,0,dikesnet
2,53,279.474004,10.0,0.408680,287.564349,1.5,0.045771,262.902795,1.5,0.925836,...,6,3,6,8,7,3,4,6,0,dikesnet
3,56,325.958606,1.5,0.783194,211.997937,1.0,0.652039,114.088627,10.0,0.662082,...,6,3,6,8,7,3,4,7,0,dikesnet
4,16,39.251177,1.0,0.657168,48.146365,10.0,0.386861,268.630431,1.0,0.992420,...,6,3,6,8,7,3,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,7,341.850761,10.0,0.421758,33.941253,1.5,0.870921,166.135860,10.0,0.514628,...,0,9,0,2,0,10,2,49,3,dikesnet
196,53,217.791879,10.0,0.171665,149.001297,1.0,0.800443,301.426344,1.0,0.221792,...,0,9,0,2,0,10,2,50,3,dikesnet
197,80,291.763640,1.5,0.283410,208.523795,1.0,0.468982,257.748560,1.0,0.956378,...,0,9,0,2,0,10,2,51,3,dikesnet
198,66,257.610455,10.0,0.549575,81.669389,1.5,0.696132,251.912594,1.5,0.789223,...,0,9,0,2,0,10,2,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.3 Expected Annual Damage,A.3 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs,Total Expected Annual Damage,Total Expected Number of Deaths
0,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.375631e+06,0.000105,0.000000e+00,0.000000,1.409218e+09,1.375631e+06,0.000105
1,0.0,0.0,7.829059e+04,0.000011,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,1.409218e+09,7.829059e+04,0.000011
2,0.0,0.0,8.377831e+07,0.010754,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,1.409218e+09,8.377831e+07,0.010754
3,0.0,0.0,8.940673e+05,0.000130,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,1.409218e+09,8.940673e+05,0.000130
4,0.0,0.0,6.519086e+06,0.000737,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,1.409218e+09,6.519086e+06,0.000737
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.616235e+06,0.000117,0.000000e+00,0.000000,1.988064e+09,1.616235e+06,0.000117
196,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.827241e+06,0.000196,6.647299e+06,0.001359,1.988064e+09,8.474540e+06,0.001555
197,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.344801e+07,0.000821,2.564575e+07,0.003349,1.988064e+09,3.909376e+07,0.004170
198,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.315388e+07,0.000974,1.454734e+06,0.000297,1.988064e+09,1.460862e+07,0.001272


In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [12]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:50<00:00,  5.95it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [13]:
experiments, outcomes = results

In [14]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.3 Expected Annual Damage,A.3 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs,Total Expected Annual Damage,Total Expected Number of Deaths
0,0.000000e+00,0.000000,8.068701e+07,0.067377,1.415803e+08,0.222569,1.371293e+07,0.005733,3.984724e+08,0.311637,3.077725e+08,6.344527e+08,0.607317
1,0.000000e+00,0.000000,7.131028e+07,0.068461,1.035388e+09,1.795133,0.000000e+00,0.000000,0.000000e+00,0.000000,3.077725e+08,1.106698e+09,1.863594
2,0.000000e+00,0.000000,2.222731e+08,0.222635,9.392861e+08,1.762615,0.000000e+00,0.000000,4.846935e+06,0.004625,3.077725e+08,1.166406e+09,1.989875
3,0.000000e+00,0.000000,2.690401e+08,0.218918,0.000000e+00,0.000000,3.595198e+06,0.001479,0.000000e+00,0.000000,3.077725e+08,2.726352e+08,0.220397
4,0.000000e+00,0.000000,2.138971e+08,0.185537,1.104389e+09,1.784495,3.817439e+06,0.001492,0.000000e+00,0.000000,3.077725e+08,1.322104e+09,1.971524
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.000000e+00,0.000000,3.760541e+08,0.366145,0.000000e+00,0.000000,1.777889e+07,0.008375,2.298732e+08,0.205772,3.984984e+08,6.237062e+08,0.580293
296,6.685668e+06,0.005904,1.891072e+07,0.023605,7.230019e+06,0.016968,0.000000e+00,0.000000,8.940679e+07,0.104382,3.984984e+08,1.222332e+08,0.150860
297,2.003710e+08,0.135493,4.111340e+07,0.039939,0.000000e+00,0.000000,9.647934e+07,0.044317,0.000000e+00,0.000000,3.984984e+08,3.379637e+08,0.219749
298,5.529925e+08,0.339956,4.924554e+08,0.406227,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,3.984984e+08,1.045448e+09,0.746183


In [18]:
outcomes.keys()

dict_keys(['A.1 Expected Annual Damage', 'A.1 Expected Number of Deaths', 'A.2 Expected Annual Damage', 'A.2 Expected Number of Deaths', 'A.3 Expected Annual Damage', 'A.3 Expected Number of Deaths', 'A.4 Expected Annual Damage', 'A.4 Expected Number of Deaths', 'A.5 Expected Annual Damage', 'A.5 Expected Number of Deaths', 'Total Infrastructure Costs', 'Total Expected Annual Damage', 'Total Expected Number of Deaths'])